<a href="https://colab.research.google.com/github/gizdatalab/CPU_Tracs/blob/main/notebooks/IKI_M_Indicators_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install evaluate
! pip install codecarbon

In [ ]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

[codecarbon INFO @ 22:29:01] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 22:29:01] Energy consumed for all GPUs : 0.000040 kWh. Total GPU Power : 9.504766223017581 W
[codecarbon INFO @ 22:29:01] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:29:01] 0.000237 kWh of electricity used since the beginning.


In [ ]:
# fetching the category/indicator data from original raw data
repo_id = "GIZ/policy_classification"
filename="dataset/Step5/classification_rawfull_grouped.json"
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id, filename=filename,
                repo_type="dataset",use_auth_token=True,
local_dir= 'data/')

# filter data
data = pd.read_json('data/' + filename)\
         .query("Source == 'IKITracs'")\
         .query("Strategy == 'medium'")\
         .filter(['Context','Alpha3','Language','IkiInfo','IKI_DataSheet',
                  'IKI_Parameter','TranslatedContext'])

data['check']= data.IKI_DataSheet.apply(lambda  x: 'mitigation' in x)
data = data[data.check==True].reset_index(drop=True)
# some contexts have non english text, we use translted text fro those cases
data['Context'] = data.apply(lambda x: x['Context'] if x['Language']== 'en'
                                                    else x['TranslatedContext'],
                                                     axis=1)
# remove duplicates
data = data.groupby(['Context'], as_index = False)\
                .agg({'IKI_Parameter':list})

data['IKI_Parameter'] = data.IKI_Parameter.apply(lambda x:
                                    [temp for sublist in x for temp in sublist])
data['IKI_Parameter'] = data.IKI_Parameter.apply(lambda x: [i for i in x if i is not None])
data['check'] = data.IKI_Parameter.apply(lambda x: len(x)==0)
data = data[data.check == False].reset_index(drop=True)

# use the data mapping sheet
dir = ''
df_catmap = pd.read_csv(dir + "mitigation_cat-mapping.csv")
dict_catmap = df_catmap.set_index('Parameter')['Category'].to_dict()
data['Iki_cat'] = data.IKI_Parameter.apply(lambda x: [dict_catmap.get(i) for i in x]
                                               if isinstance(x, list) else None)
data.rename(columns = {'Context':'text','Iki_cat':'annotation'}, inplace=True)
data = data[['text','annotation']]
data['annotation'] = data.annotation.apply(lambda x: [i for i in x if i is not None])
data['annotation'] = data.annotation.apply(lambda x: [i for i in x if i != 'Labels'])

data['check'] = data.annotation.apply(lambda x: len(x)==0)
data = data[data.check == False].reset_index(drop=True)
data = data[['text','annotation']]

In [ ]:
# reading now annotated dataset
import json
path_to_folder = '/content/drive/MyDrive/Colab Notebooks/giz/IKI_Tracs/'
df = pd.read_json(path_to_folder + 'datasets/indicators_mutlilabel_classification.json')
print(len(df))
# filter validated records
df = df[df.status == 'Validated'].reset_index(drop=True)[['text','annotation']]
df = df[df.annotation.str.len() !=0].reset_index(drop=True)
df['annotation'] = df.annotation.apply(lambda x: ['Other Transport Category'
                                            if i == 'None' else i for i in x ])

158


In [ ]:
df = pd.concat([data,df], ignore_index=True)
df.drop_duplicates(subset = ['text'],inplace=True)
df = df.reset_index(drop=True)
df.info()
df.to_csv('category_annotated.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1749 entries, 0 to 1748
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        1749 non-null   object
 1   annotation  1749 non-null   object
dtypes: object(2)
memory usage: 27.5+ KB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

[codecarbon INFO @ 22:29:31] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 22:29:31] Energy consumed for all GPUs : 0.000119 kWh. Total GPU Power : 9.50231056046444 W
[codecarbon INFO @ 22:29:31] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:29:31] 0.000710 kWh of electricity used since the beginning.


Mounted at /content/drive


# Manual Inputs


In [ ]:
###### Define Path to data ############################
# Map the columns which need to be processesed. i.e column which contains 'text' for
# prediciton and label column  for ground truth.
text_column = 'text'
label_column = 'annotation'

# train-test split
test_size=0.15

In [ ]:
#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "BAAI/bge-base-en-v1.5"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "IKI-Category-multilabel_bge"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=4.5e-5
batch_size = 8
num_train_epochs= 15
weight_decay=0.055
warmup_steps = 200
gradient_accumulation_steps = 2

# Automated

## Dataset

In [ ]:
temp = list(df[label_column])
label_names = list(set([item for sublist in temp for item in sublist]))
label_names.sort()
num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print("Label Mapping",'\n')
for key,val in label2id.items(): print(key,":",val)

# create one-hot-encoded labels
df['multi'+label_column] = df[label_column].apply(lambda x: [1 if i in x  else 0
                                                            for i in label_names])

Label Mapping 

Active mobility : 0
Alternative fuels : 1
Aviation improvements : 2
Comprehensive transport planning : 3
Digital solutions : 4
Economic instruments : 5
Education and behavioral change : 6
Electric mobility : 7
Freight efficiency improvements : 8
Improve infrastructure : 9
Land use : 10
Other Transport Category : 11
Public transport improvement : 12
Shipping improvements : 13
Transport demand management : 14
Vehicle improvements : 15


In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection import IterativeStratification

# Divide the dataset into train and test dataset (using multilabel stratifier)
df = df.sample(frac=1, random_state=42)
# Convert 'label' column to a list of lists
X = df[text_column].tolist()
X = [[x] for x in X]
y = df['multi'+label_column].tolist()
y = np.array(y)
X = np.array(X)

# Create train and test
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size=test_size)

# Turn arrays back into list
X_train = X_train.tolist()
y_train = y_train.tolist()
X_test = X_test.tolist()
y_test = y_test.tolist()

# Create train and test df
test_df = pd.DataFrame({'text': X_test, 'labels': y_test})
train_df = pd.DataFrame({'text': X_train, 'labels': y_train})

# Remove the list
train_df['text'] = train_df['text'].apply(lambda x: x[0])
test_df['text'] = test_df['text'].apply(lambda x: x[0])

print('\n','Classes Representation after stratification', "Training data:",len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation after stratification', "Test data:",len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation after stratification Training data: 1510
0 . Active mobility : 110
1 . Alternative fuels : 296
2 . Aviation improvements : 39
3 . Comprehensive transport planning : 122
4 . Digital solutions : 124
5 . Economic instruments : 161
6 . Education and behavioral change : 61
7 . Electric mobility : 364
8 . Freight efficiency improvements : 112
9 . Improve infrastructure : 200
10 . Land use : 42
11 . Other Transport Category : 61
12 . Public transport improvement : 235
13 . Shipping improvements : 68
14 . Transport demand management : 83
15 . Vehicle improvements : 267

 Classes Representation after stratification Test data: 239
0 . Active mobility : 19
1 . Alternative fuels : 52
2 . Aviation improvements : 7
3 . Comprehensive transport planning : 21
4 . Digital solutions : 22
5 . Economic instruments : 28
6 . Education and behavioral change : 11
7 . Electric mobility : 64
8 . Freight efficiency improvements : 20
9 . Improve infrastructure : 35
10 . Land use : 8
11 . O

In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for i,label in enumerate(label_names):
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero( [x[i] == 1 for x in list(train_df['labels'])]))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero([x[i] == 0 for x in list(train_df['labels'])]))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())
pos_weights = [10 if i>10 else i for i in pos_weights]

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . Active mobility : 110
1 . Alternative fuels : 296
2 . Aviation improvements : 39
3 . Comprehensive transport planning : 122
4 . Digital solutions : 124
5 . Economic instruments : 161
6 . Education and behavioral change : 61
7 . Electric mobility : 364
8 . Freight efficiency improvements : 112
9 . Improve infrastructure : 200
10 . Land use : 42
11 . Other Transport Category : 61
12 . Public transport improvement : 235
13 . Shipping improvements : 68
14 . Transport demand management : 83
15 . Vehicle improvements : 267
Positive Weights 
 {'Active mobility': 6.863636363636363, 'Alternative fuels': 2.550675675675676, 'Aviation improvements': 19.358974358974358, 'Comprehensive transport planning': 6.188524590163935, 'Digital solutions': 6.088709677419355, 'Economic instruments': 4.6894409937888195, 'Education and behavioral change': 12.37704918032787, 'Electric mobility': 2.0741758241758244, 'Freight efficiency improvements': 6.741071428571429, 'Improve infrastructure': 3.775, 'Land use

`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at BAAI/bge-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Training data: 1510
Validation data: 239


Map:   0%|          | 0/1510 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

Map:   0%|          | 0/1510 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


# Training

In [ ]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

[codecarbon INFO @ 22:56:57] [setup] RAM Tracking...
[codecarbon INFO @ 22:56:57] [setup] GPU Tracking...
[codecarbon INFO @ 22:56:57] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 22:56:57] [setup] CPU Tracking...
[codecarbon WARNING @ 22:56:57] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 22:56:58] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 22:56:58] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 22:56:58] >>> Tracker's metadata:
[codecarbon INFO @ 22:56:58]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 22:56:58]   Python version: 3.10.12
[codecarbon INFO @ 22:56:58]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 22:56:58]   Available RAM : 12.675 GB
[codecarbon INFO @ 22:56:58]   CPU count: 2
[codecarbon INFO @ 22:56:58]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 22:56:58]   GPU c

In [ ]:
multi_trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
0,0.899900,0.874170,0.388889,0.027197,0.130809,0.016867,0.018828,0.016867,0.032333,0.020223,0.028035
2,0.558200,0.555181,0.511073,0.558513,0.568532,0.722892,0.735704,0.722892,0.598802,0.595896,0.617490
4,0.288300,0.455490,0.638430,0.696920,0.644441,0.744578,0.764086,0.744578,0.687430,0.690127,0.685421
6,0.160800,0.421187,0.650794,0.707086,0.658588,0.790361,0.793096,0.790361,0.713819,0.716073,0.711582
8,0.103100,0.443493,0.727679,0.752301,0.730618,0.785542,0.787517,0.785542,0.755504,0.742469,0.748724
10,0.074000,0.454797,0.735892,0.766308,0.736799,0.785542,0.791004,0.785542,0.759907,0.748978,0.752075
12,0.060500,0.447841,0.736607,0.765098,0.737911,0.795181,0.794840,0.795181,0.764774,0.754541,0.757876
14,0.054600,0.454147,0.750000,0.770781,0.751745,0.787952,0.785774,0.787952,0.768508,0.753674,0.761488


[codecarbon INFO @ 22:57:17] Energy consumed for RAM : 0.001603 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 22:57:17] Energy consumed for all GPUs : 0.021248 kWh. Total GPU Power : 66.68444779433383 W
[codecarbon INFO @ 22:57:17] Energy consumed for all CPUs : 0.014349 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:57:17] 0.037200 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:57:17] Energy consumed for RAM : 0.002257 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 22:57:18] Energy consumed for all GPUs : 0.024359 kWh. Total GPU Power : 66.65773099993683 W
[codecarbon INFO @ 22:57:18] Energy consumed for all CPUs : 0.020198 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:57:18] 0.046814 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:57:18] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 22:57:18] Energy consumed for all GPUs : 0.000290 kWh. Total GPU Power : 69.5269045698643 

TrainOutput(global_step=1410, training_loss=0.2591696414541691, metrics={'train_runtime': 1735.1658, 'train_samples_per_second': 13.054, 'train_steps_per_second': 0.813, 'total_flos': 4446872627724288.0, 'train_loss': 0.2591696414541691, 'epoch': 14.92})

In [ ]:
tracker.stop()

[codecarbon INFO @ 23:28:35] Energy consumed for RAM : 0.004723 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 23:28:35] Energy consumed for all GPUs : 0.057642 kWh. Total GPU Power : 30.049802533850556 W
[codecarbon INFO @ 23:28:35] Energy consumed for all CPUs : 0.042291 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:28:35] 0.104657 kWh of electricity used since the beginning.


0.04736975693722146

In [ ]:
multi_trainer.push_to_hub()

[codecarbon INFO @ 23:32:37] Energy consumed for RAM : 0.004389 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 23:32:37] Energy consumed for all GPUs : 0.056585 kWh. Total GPU Power : 29.065638253797694 W
[codecarbon INFO @ 23:32:37] Energy consumed for all CPUs : 0.039349 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:32:37] 0.100323 kWh of electricity used since the beginning.


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1707863822.78ddadfe6cfb.1216.0:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1707865023.78ddadfe6cfb.1216.1:   0%|          | 0.00/20.0k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/IKI-Category-multilabel_bge/commit/4360e065b8c8531277dc6385714496858826cf10', commit_message='End of training', commit_description='', oid='4360e065b8c8531277dc6385714496858826cf10', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
log_results = multi_trainer.state.log_history
print(log_results)

import json
with open("classifier_info/TAPP_mpnet_logs.json", "w") as fp:
    json.dump(log_results , fp)

[{'loss': 0.6141, 'learning_rate': 5.35947811993517e-05, 'epoch': 1.0, 'step': 627}, {'eval_loss': 0.5409209132194519, 'eval_precision-micro': 0.6632302405498282, 'eval_precision-samples': 0.7068093963722866, 'eval_precision-weighted': 0.6740037952411381, 'eval_recall-micro': 0.7360793287566743, 'eval_recall-samples': 0.7307463574189711, 'eval_recall-weighted': 0.7360793287566743, 'eval_f1-micro': 0.697758496023138, 'eval_f1-samples': 0.6957096130155899, 'eval_f1-weighted': 0.7005652103037378, 'eval_runtime': 25.8953, 'eval_samples_per_second': 43.29, 'eval_steps_per_second': 5.445, 'epoch': 1.0, 'step': 627}, {'loss': 0.4661, 'learning_rate': 4.763980551053484e-05, 'epoch': 2.0, 'step': 1254}, {'eval_loss': 0.4958369731903076, 'eval_precision-micro': 0.7208121827411168, 'eval_precision-samples': 0.7222717811477847, 'eval_precision-weighted': 0.7389811255891985, 'eval_recall-micro': 0.7581998474446987, 'eval_recall-samples': 0.7383288730300328, 'eval_recall-weighted': 0.758199847444698

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.50
y_pred = (y_prob>=thresh).bool()

[codecarbon INFO @ 23:28:52] Energy consumed for RAM : 0.004092 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 23:28:52] Energy consumed for all GPUs : 0.054705 kWh. Total GPU Power : 37.917513882335584 W
[codecarbon INFO @ 23:28:52] Energy consumed for all CPUs : 0.036692 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:28:52] 0.095489 kWh of electricity used since the beginning.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

Active mobility
[[213   7]
 [  2  17]] 

Alternative fuels
[[176  11]
 [  7  45]] 

Aviation improvements
[[229   3]
 [  0   7]] 

Comprehensive transport planning
[[214   4]
 [  9  12]] 

Digital solutions
[[210   7]
 [  5  17]] 

Economic instruments
[[203   8]
 [  5  23]] 

Education and behavioral change
[[225   3]
 [  3   8]] 

Electric mobility
[[157  18]
 [  5  59]] 

Freight efficiency improvements
[[215   4]
 [  7  13]] 

Improve infrastructure
[[187  17]
 [  5  30]] 

Land use
[[231   0]
 [  3   5]] 

Other Transport Category
[[226   2]
 [  8   3]] 

Public transport improvement
[[187  10]
 [  7  35]] 

Shipping improvements
[[224   2]
 [  2  11]] 

Transport demand management
[[221   3]
 [  9   6]] 

Vehicle improvements
[[182  10]
 [ 11  36]] 



In [ ]:
#bge
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Active mobility,0.708333,0.894737,0.790698,19.0
1,Alternative fuels,0.803571,0.865385,0.833333,52.0
2,Aviation improvements,0.700000,1.000000,0.823529,7.0
3,Comprehensive transport planning,0.750000,0.571429,0.648649,21.0
4,Digital solutions,0.708333,0.772727,0.739130,22.0
5,Economic instruments,0.741935,0.821429,0.779661,28.0
6,Education and behavioral change,0.727273,0.727273,0.727273,11.0
7,Electric mobility,0.766234,0.921875,0.836879,64.0
8,Freight efficiency improvements,0.764706,0.650000,0.702703,20.0
9,Improve infrastructure,0.638298,0.857143,0.731707,35.0
